In [1]:
import matplotlib.pyplot as plt
import wfdb
import numpy as np
import os 
import glob
import pickle
import pywt
from sklearn import preprocessing
from scipy import io
from sklearn.metrics import confusion_matrix
from scipy.signal import resample

2021-08-17 13:09:55.827870: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
def z_score(A, lead):#输入心电图维度(lead,sample)
    mean=np.sum(A,axis=-1).reshape(lead,1)/len(A[0])
    B=(A-mean)**2
    V=np.sum(B,axis=-1).reshape(lead,1)/len(B[0])
    S=V**(1/2)
    A=(A-mean)/S
    return A

In [9]:
c_t = 0.15
fz = 128
re_sample = 256

def data_process1(i,num,begin):
    heartbeat_signal = record_wavelet[:,signal_annotation.sample[i-1]-int(RR_mean[i]-c_t*fz)-begin:signal_annotation.sample[i+1]+int(2/3*RR_mean[i])-begin]
    heartbeat_signal_resample = resample(heartbeat_signal,num,axis=1)
    heartbeat_signal_resample = np.array(heartbeat_signal_resample)
    heartbeat_signal_resample = z_score(heartbeat_signal_resample,2)
    return heartbeat_signal_resample

In [10]:
Train_ecg_data = []
Train_label = []
RR_Train = []
N_label = ['N']
S_label = ['S']
V_label = ['V']
F_label = ['F']

file_name_list = glob.glob('./mit-bih-supraventricular-arrhythmia-database-1.0.0/*.dat')
file_name_pure = []
for j in range(len(file_name_list)):
    (filename_pure,extension) = os.path.splitext(file_name_list[j])
    file_name_pure.append(filename_pure)
    
for m in range(len(file_name_pure)):
    re = os.path.split(file_name_pure[m])[1]
    record = wfdb.rdrecord('./mit-bih-supraventricular-arrhythmia-database-1.0.0/' + str(re),physical=True,sampfrom=0,sampto=None,channels=[0,1])
    signal_annotation = wfdb.rdann('./mit-bih-supraventricular-arrhythmia-database-1.0.0/' + str(re),'atr',sampfrom=0,sampto=None)

    #手工特征
    RR_infor = []
    RR_index = signal_annotation.sample
    RR = np.array(RR_index-([0]+list(RR_index))[:-1])#得到RR间期数组，第三个值开始有意义，即索引为2
    RR_diff1 = np.array(RR-([0]+list(RR))[:-1])#得到RR间期的一阶差分,索引为3代表第3个R波前后的间期差分
    
    RR_diff1 = np.array((list(RR_diff1)+[0])[1:])#将diff1变成于RR对应的索引
    RR_nor=(RR-np.min(RR[3:-3]))/(np.max(RR[3:-3])-np.min(RR[3:-3]))
    RR_diff1_nor = (RR_diff1-np.min(RR_diff1[3:-3]))/(np.max(RR_diff1[3:-3])-np.min(RR_diff1[3:-3]))
    RR_infor.append(RR_nor)
    RR_infor.append(RR_diff1_nor)
    RR_infor = np.array(RR_infor)
    
    RR_expand = [0]*5+list(RR)+[0]*6
    RR_mean = []
    for k in range(len(RR)-1):
        RR_part_mean = (np.sum(RR_expand[k:k+11])-np.min(RR_expand[k:k+11]))/10
        RR_mean.append(RR_part_mean)
    RR_mean = np.array(RR_mean)

    record_wavelet = []
    for n in range(len(record.p_signal[1])):
        coeffs = pywt.wavedec(record.p_signal[:,n], 'coif4', level=8)
        #置0第10尺度近似分量
        coeffs[-9:-8] = np.zeros_like(coeffs[-9:-8])
        coeffs[-3:-2] = np.zeros_like(coeffs[-3:-2])
        coeffs[-2:-1] = np.zeros_like(coeffs[-2:-1])
        coeffs[-1:] = np.zeros_like(coeffs[-1:])
        ECG_new2 = pywt.waverec(coeffs, 'coif4')
        ECG_new2 = ECG_new2.flatten()
        record_wavelet.append(ECG_new2)
    record_wavelet = np.array(record_wavelet)
    
    for i in range(2,len(signal_annotation.sample)-1):

        if signal_annotation.symbol[i] in N_label:
            Train_ecg_data.append(data_process1(i,re_sample,0))
            Train_label.append([0])
            RR_Train.append(RR_infor[:,i])
        if signal_annotation.symbol[i] in S_label:            
            Train_ecg_data.append(data_process1(i,re_sample,0))
            Train_label.append([1])
            RR_Train.append(RR_infor[:,i])
        if signal_annotation.symbol[i] in V_label:
            Train_ecg_data.append(data_process1(i,re_sample,0))
            Train_label.append([2])
            RR_Train.append(RR_infor[:,i])
        if signal_annotation.symbol[i] in F_label:
            Train_ecg_data.append(data_process1(i,re_sample,0))
            Train_label.append([3])
            RR_Train.append(RR_infor[:,i])

Train_ecg_data = np.array(Train_ecg_data) 
Train_label = np.array(Train_label)
RR_Train = np.array(RR_Train)

print(Train_ecg_data.shape)
print(Train_label.shape)
print(RR_Train.shape)

(184261, 2, 256)
(184261, 1)
(184261, 2)


In [11]:
with open('./train_data_svdb.txt', "wb") as text1:
    pickle.dump(Train_ecg_data,text1)
with open('./train_label_svdb.txt', "wb") as text2:
    pickle.dump(Train_label,text2)
with open('./train_RR_svdb.txt', "wb") as text3:
    pickle.dump(RR_Train,text3)

with open('./train_data_svdb.txt', "rb") as text4:
    Train_ecg_data = pickle.load(text4)
with open('./train_label_svdb.txt', "rb") as text5:
    Train_label = pickle.load(text5)
with open('./train_RR_svdb.txt', "rb") as text6:
    RR_Train = pickle.load(text6)

print(Train_ecg_data.shape)
print(Train_label.shape)
print(RR_Train.shape)

(184261, 2, 256)
(184261, 1)
(184261, 2)


In [13]:
N_=0
S_=0
V_=0
F_=0
for i in range(len(Train_ecg_data)):
    if Train_label[i]==0:
        N_+=1
    if Train_label[i]==1:
        S_+=1
    if Train_label[i]==2:
        V_+=1
    if Train_label[i]==3:
        F_+=1
print(N_)
print(S_)
print(V_)
print(F_)

162123
12182
9933
23
